In [1]:
# import necessary packages
import os
import pandas
import numpy
import matplotlib.pyplot as plt
import geopandas as gpd
from pathlib import Path
# import function for querying DOE dataframes
from monte_carlo_utils import get_DOE_atb, get_DOE_barges


# set file location and name
atb_shp_path = Path('/data/MIDOSS/shapefiles/atb_2018_01.shp')
barge_shp_path = Path('/data/MIDOSS/shapefiles/barge_2018_01.shp')
doe_xls_path = Path('/data/MIDOSS/spreadsheets/MuellerTrans4-30-20.xlsx')
fac_xls_path = Path(
    '/home/rmueller/Projects/MIDOSS/marine_transport_data/Oil_Transfer_Facilities.xlsx'
)

# import facility locations
facWA = pandas.read_excel(
    fac_xls_path,
    sheet_name = 'Washington',
    usecols="B,D,J,K"
)

In [4]:
def concat_shp(ship_type):
    """
      INPUT: 
          - ship_type ["tanker", "barge", "atb", etc]: MIDOSS-name for ship type (see oil_attribution.yaml for list)
      OUTPUT: 
          - dataframe of all 2018 ship tracks for given ship_type
      requirements: 
          This script requires being run on Salish.eoas.ubc.ca, with path to shapefiles being /data/MIDOSS/shapefiles/
    """
    from dask import delayed
    
    for months in range(1,13):
        # set file location and name
        shp_path = Path(f'/data/MIDOSS/shapefiles/{ship_type}_2018_{months:02d}.shp')
        # import shapefile using geopandas
        monthly_shp = delayed(gpd.read_file(shp_path))
        if months == 1:
            print(f'creating {ship_type} shapefile for 2018, starting with January data')
            allTracks = monthly_shp
        else:
            print(f'Concatenating {ship_type} data from month {months}')
            allTracks = delayed(gpd.GeoDataFrame(
                pandas.concat([allTracks.compute(), monthly_shp.compute()])
            ))
    return allTracks.compute()

In [7]:
%%time
allTracks = {} 
allTracks_dask = concat_shp("atb")

creating atb shapefile for 2018, starting with January data
Concatenating atb data from month 2
Concatenating atb data from month 3
Concatenating atb data from month 4
Concatenating atb data from month 5
Concatenating atb data from month 6
Concatenating atb data from month 7
Concatenating atb data from month 8
Concatenating atb data from month 9
Concatenating atb data from month 10
Concatenating atb data from month 11
Concatenating atb data from month 12
CPU times: user 41.6 s, sys: 784 ms, total: 42.4 s
Wall time: 42.4 s


In [10]:
def concat_shp(ship_type):
    """
      INPUT: 
          - ship_type ["tanker", "barge", "atb", etc]: MIDOSS-name for ship type (see oil_attribution.yaml for list)
      OUTPUT: 
          - dataframe of all 2018 ship tracks for given ship_type
      requirements: 
          This script requires being run on Salish.eoas.ubc.ca, with path to shapefiles being /data/MIDOSS/shapefiles/
    """
    from dask import delayed
    
    for months in range(1,13):
        # set file location and name
        shp_path = Path(f'/data/MIDOSS/shapefiles/{ship_type}_2018_{months:02d}.shp')
        # import shapefile using geopandas
        monthly_shp = gpd.read_file(shp_path)
        if months == 1:
            print(f'creating {ship_type} shapefile for 2018, starting with January data')
            allTracks = monthly_shp
        else:
            print(f'Concatenating {ship_type} data from month {months}')
            allTracks = delayed(gpd.GeoDataFrame(
                pandas.concat([allTracks.compute(), monthly_shp])
            ))
    return allTracks.compute()

In [11]:
%%time
allTracks = {} 
allTracks_dask = concat_shp("atb")

creating atb shapefile for 2018, starting with January data
Concatenating atb data from month 2


AttributeError: 'GeoDataFrame' object has no attribute 'compute'